In [6]:
%pip install langchain_community

In [7]:
from langchain.document_loaders import WebBaseLoader

#add website data
URL = ["https://www.mapua.edu.ph/pages/academics/undergraduate/intramuros-campus/school-of-electrical-electronics-and-computer-engineering/bachelor-of-science-in-computer-engineering"]

#load the data
data = WebBaseLoader(URL)
#extract the content
content = data.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [8]:
print(content)

[Document(metadata={'source': 'https://www.mapua.edu.ph/pages/academics/undergraduate/intramuros-campus/school-of-electrical-electronics-and-computer-engineering/bachelor-of-science-in-computer-engineering', 'title': 'Bachelor of Science in Computer Engineering | Mapúa', 'description': "Elevate your career with Mapúa's computer engineering program in the Philippines, where technological innovation meets ethical application. Join us today!", 'language': 'en'}, page_content='\n\n\n\n\n\n\n\n\nBachelor of Science in Computer Engineering | Mapúa\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n')]


In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=256,chunk_overlap=50)
chunking = text_splitter.split_documents(content)

In [10]:
print(chunking)

[Document(metadata={'source': 'https://www.mapua.edu.ph/pages/academics/undergraduate/intramuros-campus/school-of-electrical-electronics-and-computer-engineering/bachelor-of-science-in-computer-engineering', 'title': 'Bachelor of Science in Computer Engineering | Mapúa', 'description': "Elevate your career with Mapúa's computer engineering program in the Philippines, where technological innovation meets ethical application. Join us today!", 'language': 'en'}, page_content='Bachelor of Science in Computer Engineering | Mapúa')]


In [11]:
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings

import os
from getpass import getpass

# get your free access token from HuggingFace and paste it here
HF_token = getpass()
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HF_token

embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key = HF_token,model_name = "BAAI/bge-base-en-v1.5"
)

In [15]:
%pip install chromadb


  Using cached chromadb-0.6.3-py3-none-any.whl.metadata (6.8 kB)
  Using cached build-1.2.2.post1-py3-none-any.whl.metadata (6.5 kB)
  Using cached chroma_hnswlib-0.7.6.tar.gz (32 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached fastapi-0.115.11-py3-none-any.whl.metadata (27 kB)
  Using cached uvicorn-0.34.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached posthog-3.19.0-py2.py3-none-any.whl.metadata (2.9 kB)
  Using cached onnxruntime-1.20.1-cp312-cp312-win_amd64.whl.metadata (4.7 kB)
  Using cached opentelemetry_api-1.30.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.30.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached opentelemetry_instrum

In [16]:
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(chunking, embeddings)

In [17]:
print(embeddings)

api_key=SecretStr('**********') model_name='BAAI/bge-base-en-v1.5' api_url=None additional_headers={}


In [18]:
retriever = vectorstore.as_retriever(search_type="mmr",search_kwargs={"k":3})
query = "What is a Neural Network?"
docs_rel = retriever.get_relevant_documents(query)
print(docs_rel)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_23300\1737496246.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs_rel = retriever.get_relevant_documents(query)
Number of requested results 20 is greater than number of elements in index 1, updating n_results = 1


[Document(metadata={'description': "Elevate your career with Mapúa's computer engineering program in the Philippines, where technological innovation meets ethical application. Join us today!", 'language': 'en', 'source': 'https://www.mapua.edu.ph/pages/academics/undergraduate/intramuros-campus/school-of-electrical-electronics-and-computer-engineering/bachelor-of-science-in-computer-engineering', 'title': 'Bachelor of Science in Computer Engineering | Mapúa'}, page_content='Bachelor of Science in Computer Engineering | Mapúa')]


In [19]:
prompt = f"""
<|system|>>
You are an AI Assistant that follows instructions extremely well.
Please be truthful and give direct answers. Please tell 'I don't know' if user query is not in context
</s>
<|user|>
{query}
</s>
<|assistant|>
"""

In [20]:
from langchain.llms import HuggingFaceHub
from langchain.chains import RetrievalQA

model = HuggingFaceHub(repo_id="HuggingFaceH4/zephyr-7b-alpha",
                       model_kwargs={"temperature":0.5,
                                     "max_new_tokens":512,
                                     "max_length":64
                                    })

qa = RetrievalQA.from_chain_type(llm=model,retriever=retriever,chain_type="stuff")
response = qa(prompt)
print(response['result'])

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_23300\2906932817.py:4: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  model = HuggingFaceHub(repo_id="HuggingFaceH4/zephyr-7b-alpha",
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_23300\2906932817.py:11: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa(prompt)
Number of requested results 20 is greater than number of elements in index 1, updating n_results = 1
C:\Users\Lenovo\anaconda3\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Bachelor of Science in Computer Engineering | Mapúa

Question: 
<|system|>>
You are an AI Assistant that follows instructions extremely well.
Please be truthful and give direct answers. Please tell 'I don't know' if user query is not in context
</s>
<|user|>
What is a Neural Network?
</s>
<|assistant|>

Helpful Answer:

A Neural Network is a type of artificial intelligence model that is designed to learn and analyze data in a way that is similar to how the human brain processes information. It is made up of interconnected nodes or "neurons" that work together to process and analyze data. Neural networks can be used to solve a wide range of problems, including pattern recognition, image classification, and natural language processing.


In [1]:
%pip install streamlit